<a href="https://colab.research.google.com/github/ALEXSILVA794/T-cnicas-de-Desenvolvimento-de-Algoritmos/blob/main/algoritimocopa2026_chaveamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#exercicio 24/10
#Alex Paulo
#Técnicas de Desenvolvimento de Algoritimos
"""
Chaveamento da FIFA — World Cup 2026 (Python)
Implementa critérios oficiais FIFA para ordenação em grupos e seleção dos 8 melhores terceiros.
Estruturas: list, dict, tuple.
Comentários passo-a-passo em português.
Referências (critério de desempate e formato): Regulamentos FIFA + anúncio do formato (48 seleções / 12 grupos x 4).
"""

from typing import List, Tuple, Dict
import random

random.seed(0)  # para reprodutibilidade de exemplo

# ----------------------------
# Tipos e estruturas (tupla/list/dict)
# ----------------------------
# Team: tupla (nome, codigo) -> ex: ("Brasil","BRA")
Team = Tuple[str, str]

# Exemplo: lista de 48 times (substitua por times reais)
teams_48: List[Team] = [
    ("Brasil","BRA"),("Argentina","ARG"),("França","FRA"),("Alemanha","GER"),
    ("Espanha","ESP"),("Inglaterra","ENG"),("Portugal","POR"),("Itália","ITA"),
    ("México","MEX"),("EUA","USA"),("Canadá","CAN"),("Japão","JPN"),
    ("Coreia do Sul","KOR"),("Austrália","AUS"),("Marrocos","MAR"),("Senegal","SEN"),
    ("Gana","GHA"),("Suíça","SUI"),("Holanda","NED"),("Bélgica","BEL"),
    ("Croácia","CRO"),("Dinamarca","DEN"),("Suécia","SWE"),("Polônia","POL"),
    ("Áustria","AUT"),("Rep.Checa","CZE"),("Turquia","TUR"),("Camarões","CMR"),
    ("Tunísia","TUN"),("Egito","EGY"),("Irã","IRN"),("Arábia Saudita","KSA"),
    ("Colômbia","COL"),("Chile","CHI"),("Uruguai","URU"),("Peru","PER"),
    ("Venezuela","VEN"),("Equador","ECU"),("Paraguai","PAR"),("Costa Rica","CRC"),
    ("Honduras","HON"),("Bolívia","BOL"),("Nova Zelândia","NZL"),("Filipinas","PHI"),
    ("Índia","IND"),("Jamaica","JAM"),("Nigéria","NGA"),("Uruguai2","URU2")
]

# ----------------------------
# 1) Sorteio: distribuir em 12 grupos de 4 (A..L)
#    - usa lista de nomes de grupos (lista)
#    - groups: dict[group_name] = list[Team] (dicionário)
# ----------------------------
group_names = [chr(ord('A') + i) for i in range(12)]  # A..L
random.shuffle(teams_48)
groups: Dict[str, List[Team]] = {}
for i, g in enumerate(group_names):
    groups[g] = teams_48[i*4:(i+1)*4]  # 4 por grupo

# ----------------------------
# 2) Estrutura de estatísticas por grupo/time (dicionários e tuplas)
#    Cada time -> stats dict: {'pts','gf','ga','gd','fair_play','matches':[...] }
# ----------------------------
def init_group_stats(group: List[Team]) -> Dict[str, dict]:
    stats = {}
    for t in group:
        code = t[1]
        stats[code] = {
            'team': t,   # tupla (nome,codigo)
            'pts': 0,
            'gf': 0,
            'ga': 0,
            'gd': 0,
            'fair_play': 0,   # pontos de fair-play (menor melhor)
            'matches': []     # lista de dicts {'opponent': code, 'gf':x, 'ga':y}
        }
    return stats

# ----------------------------
# 3) Simular jogos de grupo (round-robin dentro do grupo)
#    - para 4 times: 6 partidas (pares combinatórios)
#    - simulate_match é um stub (substitua por resultados reais)
# ----------------------------
from itertools import combinations

def simulate_match(t1: Team, t2: Team) -> Tuple[int,int]:
    """Simulação simples: gols aleatórios 0..4 (só para exemplo)."""
    return random.randint(0,4), random.randint(0,4)

def play_group(group: List[Team]) -> Dict[str, dict]:
    st = init_group_stats(group)
    for a,b in combinations(range(len(group)), 2):
        t1 = group[a]; t2 = group[b]
        c1 = t1[1]; c2 = t2[1]
        g1, g2 = simulate_match(t1, t2)
        # atualizar gols e saldo
        st[c1]['gf'] += g1; st[c1]['ga'] += g2; st[c1]['gd'] = st[c1]['gf'] - st[c1]['ga']
        st[c2]['gf'] += g2; st[c2]['ga'] += g1; st[c2]['gd'] = st[c2]['gf'] - st[c2]['ga']
        # pontos
        if g1 > g2:
            st[c1]['pts'] += 3
        elif g2 > g1:
            st[c2]['pts'] += 3
        else:
            st[c1]['pts'] += 1; st[c2]['pts'] += 1
        # registro confronto direto (para desempate H2H)
        st[c1]['matches'].append({'opponent': c2, 'gf': g1, 'ga': g2})
        st[c2]['matches'].append({'opponent': c1, 'gf': g2, 'ga': g1})
        # fair-play (exemplo: 0..3)
        st[c1]['fair_play'] += random.randint(0,3)
        st[c2]['fair_play'] += random.randint(0,3)
    return st

# jogar todos os grupos
standings_all: Dict[str, Dict[str, dict]] = {}
for gname, teams in groups.items():
    standings_all[gname] = play_group(teams)

# ----------------------------
# 4) Função que aplica os critérios FIFA de desempate em um grupo
#    Ordem (conforme regulamento FIFA):
#    a) pts (total)
#    b) gd (total)
#    c) gf (total)
#    se empate entre >=2:
#      d) pts no confronto direto (entre os empatados)
#      e) gd no confronto direto
#      f) gf no confronto direto
#      g) (em competições com "away" aplica away goals entre os empatados) -- para torneio final geralmente não relevante
#      h) fair-play (maior pontuação de conduta = melhor) -> na prática usamos menor dedução
#      i) sorteio (drawing of lots)
#    (implementação aproximada e fiel aos passos principais)
# ----------------------------
def h2h_stats(team_code: str, block_codes: List[str], group_stats: Dict[str, dict]):
    """Calcula pts_h2h, gd_h2h, gf_h2h apenas considerando partidas entre block_codes."""
    pts = gf = ga = 0
    for m in group_stats[team_code]['matches']:
        if m['opponent'] in block_codes:
            gf += m['gf']; ga += m['ga']
            if m['gf'] > m['ga']:
                pts += 3
            elif m['gf'] == m['ga']:
                pts += 1
    gd = gf - ga
    return pts, gd, gf

def break_ties_fifa(group_stats: Dict[str, dict]) -> List[str]:
    """
    Retorna lista de códigos de times ordenada 1..4 aplicando critérios FIFA.
    Comentários passo-a-passo dentro da função.
    """
    # 1) ordena por pts, gd, gf (critérios a, b, c)
    items = list(group_stats.values())
    items.sort(key=lambda s: (s['pts'], s['gd'], s['gf']), reverse=True)

    ordered = []
    i = 0
    while i < len(items):
        # detecta bloco de empate estrito em pts/gd/gf
        j = i + 1
        block = [items[i]]
        while j < len(items) and items[j]['pts'] == items[i]['pts'] and items[j]['gd'] == items[i]['gd'] and items[j]['gf'] == items[i]['gf']:
            block.append(items[j]); j += 1

        if len(block) == 1:
            ordered.append(block[0]['team'][1])
        else:
            # 2) aplica critérios de confronto direto (d,e,f)
            codes = [b['team'][1] for b in block]
            h2h_list = []
            for b in block:
                code = b['team'][1]
                pts_h, gd_h, gf_h = h2h_stats(code, codes, group_stats)
                # armazenar fair_play para desempate posterior
                h2h_list.append((code, pts_h, gd_h, gf_h, group_stats[code]['fair_play']))
            # ordenar por h2h pts/gd/gf
            h2h_list.sort(key=lambda x: (x[1], x[2], x[3]), reverse=True)

            # podem ainda existir sub-empates; aplicar fair-play (menor é melhor)
            k = 0
            while k < len(h2h_list):
                l = k + 1
                sub = [h2h_list[k]]
                while l < len(h2h_list) and h2h_list[l][1] == h2h_list[k][1] and h2h_list[l][2] == h2h_list[k][2] and h2h_list[l][3] == h2h_list[k][3]:
                    sub.append(h2h_list[l]); l += 1
                if len(sub) == 1:
                    ordered.append(sub[0][0])
                else:
                    # aplicar fair-play: menor dedução (i.e., menor value) é melhor
                    sub.sort(key=lambda x: x[4])  # fair_play ascendente
                    m = 0
                    while m < len(sub):
                        n = m + 1
                        eq = [sub[m]]
                        while n < len(sub) and sub[n][4] == sub[m][4]:
                            eq.append(sub[n]); n += 1
                        if len(eq) == 1:
                            ordered.append(eq[0][0])
                        else:
                            # se persistir empate -> sorteio (random)
                            codes_eq = [e[0] for e in eq]
                            random.shuffle(codes_eq)
                            ordered.extend(codes_eq)
                        m = n
                k = l
        i = j
    return ordered

# aplicar ordenação por grupo
group_rankings: Dict[str, List[str]] = {}
for gname, gstats in standings_all.items():
    ordered_codes = break_ties_fifa(gstats)
    group_rankings[gname] = ordered_codes  # lista de códigos 1..4

# ----------------------------
# 5) Seleção dos classificados:
#    - avançam 1º e 2º de cada grupo => 24 times
#    - mais os 8 melhores terceiros entre os 12 grupos => total 32
#    Critério para "melhores terceiros": pontos, gd, gf, fair-play, sorteio (aplicar igual comparador)
# ----------------------------
# construir lista dos terceiros
thirds = []
for g in group_names:
    code = group_rankings[g][2]  # índice 2 = 3º lugar
    st = standings_all[g][code]
    thirds.append({'group': g, 'code': code, 'pts': st['pts'], 'gd': st['gd'], 'gf': st['gf'], 'fair_play': st['fair_play'], 'team': st['team']})

# ordenar terceiros por pts, gd, gf, fair_play (menor fair_play melhor)
thirds.sort(key=lambda x: (x['pts'], x['gd'], x['gf'], -x['fair_play']), reverse=True)
best_8_thirds = thirds[:8]  # os 8 melhores terceiros classificados

# criar lista de classificados (1º e 2º de cada grupo) + melhores 8 terceiros
qualified = []
for g in group_names:
    first = standings_all[g][group_rankings[g][0]]['team']
    second = standings_all[g][group_rankings[g][1]]['team']
    qualified.append({'slot': f"{g}1", 'team': first})
    qualified.append({'slot': f"{g}2", 'team': second})
# adicionar terceiros (colocar como 3A..3L conforme venceu o critério)
for t in best_8_thirds:
    qualified.append({'slot': f"{t['group']}3", 'team': t['team']})

# ----------------------------
# 6) Montagem do Round of 32 (R32)
#    - O mapa oficial que determina quem joga contra quem quando os terceiros classificados
#      são determinados está no ANNEX C dos Regulamentos (contém 495 combinações).
#    - Aqui: mostramos como aplicar o mapa oficial se você carregar um dicionário/matriz AnnexC.
# ----------------------------
# Exemplo simplificado: se Annex C não for carregado, usamos um mapeamento demonstrativo.
# Para usar o mapa real: baixe o "Regulations" / Annex C da FIFA e construa uma estrutura dict
# que, para cada combinação de grupos_terceiros (ex: tuple(sorted(['A','C','D',...])) ) retorna o emparelhamento.
#
# placeholder_map: exemplo didático (NÃO é o mapa oficial)
placeholder_r32 = [
    # (slot1, slot2) -- slot strings correspondem a "A1", "B2", "C1", etc.
    ("A1","B2"), ("C1","D2"), ("E1","F2"), ("G1","H2"),
    ("I1","J2"), ("K1","L2"), ("B1","A2"), ("D1","C2"),
    ("F1","E2"), ("H1","G2"), ("J1","I2"), ("L1","K2"),
    # adiciona confrontos para alguns terceiros: (exemplo fictício)
    ("1A","3B"), # OBS: ex.: "1A" significa vencedor do grupo A; modelagem real exige mapear conforme Annex C
]

# função para construir partidas R32 a partir dos slots (necessita map real para casos com 3ºs)
def build_r32_matches(qualified_slots: List[dict], mapping):
    """
    qualified_slots: lista de dict {'slot': 'A1'/'B2'/'C3', 'team': ('Nome','COD')}
    mapping: lista de pares (slotA, slotB) ou um dict que contenha as combinações oficiais (Annex C).
    Retorna lista de partidas (teamA, teamB).
    """
    # construir lookup slot -> team
    lookup = {q['slot']: q['team'] for q in qualified_slots}
    matches = []
    for s1, s2 in mapping:
        # suportar chaves no formato 'A1','B2' ou '1A' dependendo do mapping
        if s1 in lookup and s2 in lookup:
            matches.append((lookup[s1], lookup[s2]))
        else:
            # se mapping envolve placeholders para terceiros, precisa mapear usando Annex C
            # aqui apenas ignoramos
            pass
    return matches

# construir R32 (exemplo)
r32_matches = build_r32_matches(qualified, placeholder_r32)

# ----------------------------
# 7) Output resumido (quem se classificou e exemplo de jogos R32)
# ----------------------------
print("== Exemplos: primeiros colocados de cada grupo ==")
for g in group_names:
    code = group_rankings[g][0]
    t = standings_all[g][code]['team']
    print(f"Grupo {g} - 1º: {t[0]} ({t[1]})")

print("\n== Oito melhores terceiros (exemplo) ==")
for t in best_8_thirds:
    print(f"{t['group']} - {t['team'][0]} ({t['code']}) pts:{t['pts']} gd:{t['gd']} gf:{t['gf']}")

print("\n== Exemplos de partidas R32 (com mapping demonstrativo) ==")
for a,b in r32_matches:
    print(f"{a[0]} ({a[1]})  vs  {b[0]} ({b[1]})")

# ============================
# Observações finais / Como usar com o mapa oficial (Annex C)
# - Baixe o documento "Regulations FIFA World Cup 2026" (procure por Annex C — combinações dos 8 terceiros).
#   A partir dele construa um dict que, para cada conjunto de grupos dos terceiros classificados,
#   indique exatamente quais slots (ex.: 1A vs 3C ; 1B vs 3F ; ...) devem ser usados.
# - No código acima substitua `placeholder_r32` pelo mapping derivado do Annex C e chame build_r32_matches().
# - As regras de desempate aplicadas aqui seguem os passos essenciais do regulamento FIFA:
#   pontos -> saldo -> gols -> confronto direto entre empatados -> saldo/gols H2H -> fair-play -> sorteio.
# ============================


SyntaxError: ':' expected after dictionary key (ipython-input-1046799674.py, line 8)

In [ ]:
#Dicionário